# LOWER BOUND (LB-NEW1)

## PREPARING THE ENVIRONMENT

Installing the relevant libraries:

In [1]:
import os
import mip
import numpy as np
import pandas as pd

from collections import defaultdict

Defining the constants:

In [2]:
BIN_SIZE = [100,
            120,
            150]

MAX_COST = BIN_SIZE[-1]

Defining classes and functions:

In [3]:
class Instance:
    def __init__(self, filename):
        self.n = None
        self.v = None
        self.G = None

        with open(filename, 'r') as file:
            self.n = int(file.readline().strip())
            self.v = np.zeros(self.n, dtype=int)
            self.G = np.zeros((self.n, self.n), dtype=int)

            for line in file:
                tokens = list(map(int, line.split()))
                if not tokens:
                    continue

                v1 = tokens[0]
                self.v[v1] = tokens[1]

                for v2 in tokens[2:]:
                    self.G[v1][v2] = 1
                    self.G[v2][v1] = 1

        for v1 in range(self.n - 1):
            for v2 in range(v1 + 1, self.n):
                if self.v[v1] + self.v[v2] > MAX_COST:
                    self.G[v1][v2] = 1
                    self.G[v2][v1] = 1

    def __getitem__(self, i):
        return self.v[i]

    def describe(self):
        print(f'Number of items = {self.n}')

        print('v =')
        print(self.v)
        print('G =')
        print(self.G)

Reading an instance:

In [4]:
filename = '../instances/train/Correia_Random_1_3_6_4.txt'

instance = Instance(filename)

instance.describe()

Number of items = 100
v =
[ 81  82  67  64  90  56  56 100  85  72  57  73  99  67  57  75  53  65
  50  62 100  78  80  97 100  96  73  75  51  92  86  79  93  91  50  59
  91  83  97  59  51  77  93  68  56  74  78  54  53  67  78  54  73  85
  79  82  91  84  97  91  92  71  78  76  90  52  52  70  80  85  88  82
  83  87  65  89  75  78  62  74  88  54  84  62  94  76  60  64  94  63
  56  68  90  88  69  89 100  95  51  76]
G =
[[0 1 1 ... 1 0 1]
 [1 0 1 ... 1 1 1]
 [1 1 0 ... 1 0 1]
 ...
 [1 1 1 ... 0 1 1]
 [0 1 0 ... 1 0 1]
 [1 1 1 ... 1 1 0]]


Loading heuristic upper bounds:

In [5]:
data_train = pd.read_csv('../out/results_train_final.txt', sep='\s+')
data_test  = pd.read_csv('../out/results_test_final.txt' , sep='\s+')

data = pd.concat([data_train, data_test])

instance2heuristic = dict(zip(data['instance'], data['obj_lnsa']))

## GETTING A MAXIMUM CLIQUE

Defining a function to obtain a maximal clique using the algorithm described by Johnson:

In [6]:
def johnson_maximal_clique(instance):
    S = set()
    R = set(range(instance.n))

    while R:
        y = max(R, key=lambda v: sum(instance.G[v][w] for w in R))

        S.add(y)
        R = {w for w in R if instance.G[y][w] == 1}

    return S

def verify_maximal_clique(clique):
    for v in clique:
        for w in clique:
            if v != w and not instance.G[v][w]:
                return False

    return True

Getting a maximum clique:

In [7]:
clique = johnson_maximal_clique(instance)

print('Maximum clique =', clique)

Maximum clique = {0, 1, 4, 7, 8, 12, 15, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 36, 37, 38, 41, 42, 46, 50, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 68, 69, 70, 71, 72, 73, 75, 76, 77, 80, 82, 84, 85, 88, 92, 93, 95, 96, 97, 99}


Checking for maximal clique:

In [8]:
if verify_maximal_clique(clique):
    print('The maximum clique obtained is valid.')
else:
    print('The maximum clique obtained is not valid.')

The maximum clique obtained is valid.


## LOWER BOUND

Getting relevant information for models:

In [9]:
def get_params(instance, clique, bins):
    n = instance.n
    V = instance.v
    G = instance.G

    B  = list(range(len(bins)))
    Bi = {
        i : list(k for k in B if V[i] <= bins[k])
        for i in clique
    }

    Vc  = clique
    Vc_ = set(range(n)) - clique
    Vi  = {
        i : list(j for j in Vc_ if G[i][j] == 0)
        for i in Vc
    }

    return n, V, G, B, Bi, Vc, Vc_, Vi


n, V, G, B, Bi, Vc, Vc_, Vi = get_params(instance, clique, BIN_SIZE)

### CALCULATING IP-U(k)

Defining a function to calculate IP-U(k):

In [10]:
def ip_uk(instance, clique_comp, Wk):
    n = instance.n
    V = instance.v
    G = instance.G

    model = mip.Model(sense=mip.MAXIMIZE)
    model.verbose = 0

    x = {i: model.add_var(var_type=mip.BINARY) for i in clique_comp}

    model.objective = mip.xsum(V[i] * x[i] for i in x)

    model += mip.xsum(V[i] * x[i] for i in x) <= Wk
    for i1 in x:
        for i2 in x:
            if i1 != i2 and G[i1][i2]:
                model += x[i1] + x[i2] <= 1

    status = model.optimize()
    obj    = model.objective_value

    return status, int(obj)

Calculating the effective space for each bin type:

In [11]:
IPU = [0 for _ in B]

for k in B:
    status, obj = ip_uk(instance, Vc_, BIN_SIZE[k])

    if status == mip.OptimizationStatus.OPTIMAL:
        IPU[k] = obj
    else:
        print(f'Infeasibility in type bin {k}!')

for k, ipuk in enumerate(IPU):
    print(f'Bin of type {k} =', ipuk)

Bin of type 0 = 74
Bin of type 1 = 120
Bin of type 2 = 148


### CALCULATING IP-U(i, k)

Defining a function to calculate IP-U(i, k):

In [12]:
def ip_uik(instance, vi, i, Wk):
    n = instance.n
    V = instance.v
    G = instance.G

    model = mip.Model(sense=mip.MAXIMIZE)
    model.verbose = 0

    x = {j: model.add_var(var_type=mip.BINARY) for j in vi}

    model.objective = mip.xsum(V[j] * x[j] for j in x)

    model += mip.xsum(V[j] * x[j] for j in x) <= Wk - V[i]
    for j1 in x:
        for j2 in x:
            if j1 != j2 and G[j1][j2]:
                model += x[j1] + x[j2] <= 1

    status = model.optimize()
    obj    = model.objective_value

    return status, int(obj)

Calculating the effective space for each pair of item in the clique and bin type:

In [13]:
R = []
W = defaultdict(dict)

for i in Vc:
    vi = Vi[i]

    if vi:
        for k in Bi[i]:
            status, obj = ip_uik(instance, vi, i, BIN_SIZE[k])

            if status == mip.OptimizationStatus.OPTIMAL:
                if obj or not W[i]:
                    W[i][k] = obj
                else:
                    R.append((i, k))
            else:
                print(f'Infeasibility in item {i} and type bin {k}!')
    else:
        k = Bi[i][0]

        Bi[i] = [k]
        W[i][k] = 0

for i, k in R:
    Bi[i].remove(k)

for i in list(W.keys())[:5]:
    for k in W[i].keys():
        print(f'Item {i} and type bin {k} =', W[i][k])

Item 0 and type bin 0 = 0
Item 0 and type bin 2 = 69
Item 1 and type bin 0 = 0
Item 1 and type bin 2 = 68
Item 4 and type bin 0 = 0
Item 4 and type bin 2 = 59
Item 7 and type bin 0 = 0
Item 7 and type bin 2 = 50
Item 8 and type bin 0 = 0
Item 8 and type bin 2 = 65


### LOWER BOUND

Defining the integer linear programming model to the lower bound:

In [14]:
def lower_bound(instance, B, Bi, Vc, Vc_, Vi, BIN_SIZE, IPU, W, time_limit=60, heuristic=None):
    n = instance.n
    V = instance.v
    G = instance.G

    model = mip.Model(sense=mip.MINIMIZE)
    model.verbose = 0

    x  = {(i, k): model.add_var(var_type=mip.BINARY)
          for i in Vc
          for k in Bi[i]}
    f  = {(j, i): model.add_var(lb=0)
          for i in Vc
          for j in Vi[i]}
    f0 = {j: model.add_var(lb=0) for j in Vc_}
    y  = {k: model.add_var(var_type=mip.INTEGER, lb=0) for k in B}

    model.objective = (
        mip.xsum(BIN_SIZE[k] * y[k] for k in B) +
        mip.xsum(BIN_SIZE[k] * x[i, k] for i in Vc for k in Bi[i])
    )

    if heuristic:
        model += model.objective <= heuristic

    for i in Vc:
        model += mip.xsum(f[j, i] for j in Vi[i]) <= \
                 mip.xsum(W[i][k] * x[i, k] for k in Bi[i])

    for i in Vc:
        model += mip.xsum(x[i, k] for k in Bi[i]) == 1

    for j in Vc_:
        model += f0[j] + mip.xsum(f[j, i] for i in Vc if not G[i][j]) == V[j]

    model += mip.xsum(f0[j] for j in Vc_) <= mip.xsum(IPU[k] * y[k] for k in B)

    status = model.optimize(max_seconds=time_limit)
    obj    = model.objective_value

    return status, int(obj), (x, f, f0, y)

In [15]:
h = instance2heuristic[filename.lstrip('../')]


status, obj, mvars = lower_bound(instance, B, Bi, Vc, Vc_, Vi,
                                 BIN_SIZE, IPU, W, heuristic=h)

print('Objetive value =', obj)

Objetive value = 7750


## APPLYING TO ALL INSTANCES WITH 100 ITEMS

Getting the paths to all instances:

In [16]:
filename1 = '../instances/train'
filename2 = '../instances/test'


files = [os.path.join(filename1, f) for f in os.listdir(filename1) if f.startswith('Correia_Random_1')] + \
        [os.path.join(filename2, f) for f in os.listdir(filename2) if f.startswith('Correia_Random_1')]

print('Number of instances =', len(files))

Number of instances = 360


Getting the lower bounds of all instances:

In [17]:
out = open('../out/lower_bounds_01.txt', 'a')

for idx, filename in enumerate(files):
    print(f"{idx + 1}. {filename.split('/')[-1].split('.')[0]}", end=' ')

    instance = Instance(filename)

    clique = johnson_maximal_clique(instance)
    n, V, G, B, Bi, Vc, Vc_, Vi = get_params(instance, clique, BIN_SIZE)

    # Calculating the effective space for each bin type
    IPU = [0 for _ in B]
    for k in B:
        status, obj = ip_uk(instance, Vc_, BIN_SIZE[k])

        if status == mip.OptimizationStatus.OPTIMAL:
            IPU[k] = obj
        else:
            print(f'Inviabilidade no bin de tipo {k}!')

    # Calculating the effective space for each pair of item in the clique and bin type
    R = []
    W = defaultdict(dict)

    for i in Vc:
        vi = Vi[i]

        if vi:
            for k in Bi[i]:
                status, obj = ip_uik(instance, vi, i, BIN_SIZE[k])

                if status == mip.OptimizationStatus.OPTIMAL:
                    if obj or not W[i]:
                        W[i][k] = obj
                    else:
                        R.append((i, k))
                else:
                    print(f'Infeasibility in item {i} and type bin {k}!')
        else:
            k = Bi[i][0]

            Bi[i] = [k]
            W[i][k] = 0

    for i, k in R:
        Bi[i].remove(k)

    # Lower bound
    filename = filename.lstrip('../')

    h = instance2heuristic[filename]
    status, obj, mvars = lower_bound(instance, B, Bi, Vc, Vc_, Vi,
                                     BIN_SIZE, IPU, W, heuristic=h)

    gap = (h - obj) / obj * 100

    print(f'{obj} - {h} ({gap:.2f})')
    out.write(f'{filename} {obj}\n')

out.close()

1. Correia_Random_1_2_4_9 6070 - 6080 (0.16)
2. Correia_Random_1_2_0_9 6060 - 6070 (0.17)
3. Correia_Random_1_1_5_9 4840 - 4850 (0.21)
4. Correia_Random_1_1_6_8 5290 - 5320 (0.57)
5. Correia_Random_1_3_4_6 7820 - 7950 (1.66)
6. Correia_Random_1_2_5_7 6060 - 6090 (0.50)
7. Correia_Random_1_3_4_8 7510 - 7590 (1.07)
8. Correia_Random_1_1_3_3 4900 - 4910 (0.20)
9. Correia_Random_1_2_1_4 6240 - 6250 (0.16)
10. Correia_Random_1_3_2_6 7800 - 7950 (1.92)
11. Correia_Random_1_3_5_3 7760 - 7860 (1.29)
12. Correia_Random_1_2_5_2 6020 - 6050 (0.50)
13. Correia_Random_1_3_0_3 7740 - 7850 (1.42)
14. Correia_Random_1_2_5_1 5900 - 5930 (0.51)
15. Correia_Random_1_1_9_2 5780 - 5960 (3.11)
16. Correia_Random_1_3_1_1 7460 - 7530 (0.94)
17. Correia_Random_1_3_8_5 7320 - 7450 (1.78)
18. Correia_Random_1_1_6_2 5610 - 5660 (0.89)
19. Correia_Random_1_3_3_9 7440 - 7550 (1.48)
20. Correia_Random_1_1_0_4 4610 - 4610 (0.00)
21. Correia_Random_1_2_4_8 5600 - 5620 (0.36)
22. Correia_Random_1_2_0_10 5300 - 5300 (0.